In [117]:
import tqdm
import datetime

In [21]:
import itertools
import json
from collections import Counter
from typing import Iterable

from models import Node, Link, Response


def reformat_fullname(fullname: str) -> str:
    fullname = fullname.split()
    name = f"{fullname[0]} {fullname[1][0]}.{fullname[2][0]}."
    return name


def flatten(t: Iterable):
    return [item for sublist in t for item in sublist]

In [105]:
data = cabinet.parse_projects()
data = cabinet.filter_by_working(data)

for p in tqdm.tqdm(data):
    pid = p['id']
    p['detailed_team'] = cabinet.project_team(pid)
    
    detailed_info = cabinet.detailed_project_info(pid)
    p['projectIndustryLabel'] = detailed_info.get("projectIndustryLabel", "Неизвестно")
    p['typeDesc'] = detailed_info.get("typeDesc", "Неизвестно")
    p['leaders'] = detailed_info.get("leaders")


100%|█████████████████████████████████████████████████████████████████████████████████| 349/349 [03:14<00:00,  1.79it/s]


In [251]:
def process_member(member):
    member_clean = {}
    
    fname = member['first_name']
    if fname:
        fname = fname[0]
        
    mname = member['middle_name']
    if mname:
        mname = mname[0]
    
    joined_team = member['startDate']
    if joined_team is not None:
        in_team_duration = datetime.datetime.now() - datetime.datetime.strptime(joined_team, "%d.%m.%Y")
        in_team_duration = in_team_duration.days
    else:
        in_team_duration = None
        
    member_clean['in_team_duration'] = in_team_duration
    member_clean['name'] = f"{member['last_name']} {fname}.{mname}."
#     member_clean['name'] = member['name']
    member_clean['id'] = member['id']
    member_clean['study_group'] = member['group']
    
    return member_clean

def process_leaders(leader):
    leader_clean = {}
    
    fname = leader['first_name']
    if fname:
        fname = fname[0]
        
    mname = leader['middle_name']
    if mname:
        mname = mname[0]

    leader_clean['name'] = f"{leader['last_name']} {fname}.{mname}."
#     leader_clean['name'] = leader['fio']
    leader_clean['id'] = leader['id']
    leader_clean['role'] = 'Руководитель направления'
    
    return leader_clean

In [252]:
%%time

nodes_groups = {}
group_by = "projectIndustryLabel"
groups_labels = {}
project_members = []

clean_data = []
nodes_ids = []
for project in data:
    clean_project = {}
    
    group = project[group_by]
    if group not in groups_labels:
        groups_labels[group] = len(groups_labels)

    clean_project['group'] = groups_labels[group]
    clean_project['group_label'] = group
    
    team = project['detailed_team']
    team_clean = [process_member(member) for member in team]
    
    for m in team_clean:
        nodes_ids.append(m['id'])
        
    clean_project['team'] = team_clean
    
    leaders_clean = [process_leaders(leader) for leader in project['leaders']]
    for l in leaders_clean:
        nodes_ids.append(l['id'])
        
    clean_project['leaders'] = leaders_clean

    clean_data.append(clean_project)

CPU times: user 17.5 ms, sys: 0 ns, total: 17.5 ms
Wall time: 15.8 ms


In [270]:
leaders_clean

[{'name': 'Кофанов Ю.Н.', 'id': 35, 'role': 'Руководитель направления'},
 {'name': 'Самбурский Л.М.', 'id': 57, 'role': 'Руководитель направления'},
 {'name': ' ..', 'id': 0, 'role': 'Руководитель направления'}]

In [255]:
nodes_weight = Counter(nodes_ids)

In [289]:
nodes_weight[33]

24

In [258]:
from dataclasses import dataclass


In [272]:
@dataclass
class Node:
    id: int
    label: str
    weight: int

@dataclass
class Edge:
    from_id: int
    to_id: int
    group: int
    weight: int

In [296]:
%%time

nodes = []
edges = []
added_nodes = set()
for p in clean_data:
    #make nodes
    members_nodes = []
    
    team_nodes = []
    for m in p['team']:
        if m['id'] in added_nodes:
            continue
        added_nodes.add(m['id'])
        
        node = Node(id=m['id'], label=m['name'], weight=nodes_weight[m['id']])
        members_nodes.append(node)
        
    nodes += members_nodes
    
    
    for l in p['leaders']:
        if l['id'] in added_nodes:
            continue
            
        added_nodes.add(l['id'])
        node = Node(id=l['id'], label=l['name'], weight=nodes_weight[l['id']])
        nodes.append(node)
        
    
    
    #make edges
    for node1, node2 in itertools.combinations(members_nodes, r=2):
        edge = Edge(from_id=node1.id, to_id=node2.id, group=p['group'], weight=1)
        edges.append(edge)
        
    for member_node in p['team']:
        for leader_node in p['leaders']:
            edge = Edge(from_id=member_node['id'], 
                        to_id=leader_node['id'], 
                        group=p['group'], 
                        weight=member_node['in_team_duration'])
            
            if edge.from_id == 0 or edge.to_id == 0 or edge.from_id == edge.to_id:
                continue
            edges.append(edge)
            
    
nodes = [i for i in nodes if i.id != 0]

CPU times: user 8.98 ms, sys: 0 ns, total: 8.98 ms
Wall time: 8.41 ms


In [303]:
from pyvis.network import Network
import pickle

In [297]:
[i for i in nodes if "Королев" in i.label]

[Node(id=34, label='Королев П.С.', weight=4),
 Node(id=33, label='Королев Д.А.', weight=24),
 Node(id=1450, label='Королев К..', weight=1),
 Node(id=2190, label='Королев Д.А.', weight=1)]

In [298]:
"Королев Д.А."

'Королев Д.А.'

In [315]:
graph = Network(notebook=False, height='1000px', width='100%', bgcolor='#222222', font_color='white')
graph.barnes_hut()
graph.inherit_edge_colors(False)

for node in nodes:
    graph.add_node(node.id, label=node.label, value=node.weight)
    
for edge in edges:
    graph.add_edge(edge.from_id, edge.to_id, group=edge.group, color='red')

In [316]:
graph.save_graph("../src/graph.html")

In [311]:
with open("graph.pickle", "wb") as f:
    pickle.dumps(graph, f)

TypeError: 'Network' object cannot be interpreted as an integer

In [309]:
graph

<class 'pyvis.network.Network'> |N|=1073 |E|=2,847

In [302]:
graph.show("asd.html")